In [0]:
import sys
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision.transforms as transforms
import torchvision.datasets as dset
from torchsummary import summary

In [0]:
print("--sys.version--")
print(sys.version)

print("\n--torch.__version__--")
print(torch.__version__)

--sys.version--
3.6.8 (default, Jan 14 2019, 11:02:34) 
[GCC 8.0.1 20180414 (experimental) [trunk revision 259383]]

--torch.__version__--
1.1.0


In [0]:
batch_size = 20
total_epoch = 50
learning_rate = 0.01
use_cuda = torch.cuda.is_available()
criterion = nn.CrossEntropyLoss()

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [0]:
train_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/train", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))
test_dataset = dset.ImageFolder(root="/content/gdrive/My Drive/Colab Notebooks/pytorch/gender classification/test", transform=transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True)
test_loader = torch.utils.data.DataLoader(dataset=test_dataset, batch_size=batch_size, shuffle=False)

In [0]:
def train(model, train_loader):
  model.train()
  
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
  losses = []
  for i, (image, label) in enumerate(train_loader):
    
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
    pred_label = model(image)
    loss = criterion(pred_label, label)
    losses.append(loss.item())
    
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
  avg_loss = sum(losses)/len(losses)
  return avg_loss

In [0]:
def eval(model, test_loader):
  model.eval()
  device = next(model.parameters()).device.index
  
  total_cnt = 0
  correct_cnt = 0
  
  for i, (image, label) in enumerate(test_loader):
    if use_cuda:
      image = image.cuda()
      label = label.cuda()
      
      out = model(image)
      _, pred_label = torch.max(out.data, 1)
      total_cnt += image.data.size()[0]
      correct_cnt += (pred_label == label.data).sum().item()
      
    return correct_cnt / total_cnt

In [0]:
class SimpleMLP(nn.Module):
  def __init__(self):
    super(SimpleMLP, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.ReLU()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.ReLU()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.ReLU()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.ReLU()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
class SimpleMLP_Sigmoid(nn.Module):
  def __init__(self):
    super(SimpleMLP_Sigmoid, self).__init__()
    self.fc1 = nn.Linear(3*32*32, 8*28*28) 
    self.act1 = nn.Sigmoid()
    self.fc2 = nn.Linear(8*28*28, 8*24*24)
    self.act2 = nn.Sigmoid()    
    self.fc3 = nn.Linear(8*24*24, 16*8*8)
    self.act3 = nn.Sigmoid()
    self.fc4 = nn.Linear(16*8*8, 16*4*4)
    self.act4 = nn.Sigmoid()
    
    # Output layer
    self.out = nn.Linear(16*4*4, 10)
    
  def forward(self, x):
    x = x.view(-1, 3*32*32)
    x = self.act1(self.fc1(x))
    x = self.act2(self.fc2(x))
    x = self.act3(self.fc3(x))
    x = self.act4(self.fc4(x))
    
    out = self.out(x)
    return out

In [0]:
mlp_model = SimpleMLP().cuda()
train_loss_lst = []
test_accuracy_lst = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(mlp_model, input_size = (3, 32, 32))

1 loss : 2.2472915172576906
Accuracy : 0.5
2 loss : 2.1922542572021486
Accuracy : 0.5
3 loss : 2.1387096405029298
Accuracy : 0.5
4 loss : 2.0799371719360353
Accuracy : 0.5
5 loss : 2.0097891807556154
Accuracy : 0.5
6 loss : 1.92032151222229
Accuracy : 0.5
7 loss : 1.7975454092025758
Accuracy : 0.5
8 loss : 1.6306595087051392
Accuracy : 0.5
9 loss : 1.4105317354202271
Accuracy : 0.5
10 loss : 1.174509835243225
Accuracy : 0.5
11 loss : 0.9850618481636048
Accuracy : 0.5
12 loss : 0.8652038931846618
Accuracy : 0.5
13 loss : 0.7962158799171448
Accuracy : 0.5
14 loss : 0.7613142132759094
Accuracy : 0.6
15 loss : 0.7226347208023072
Accuracy : 0.8
16 loss : 0.7224994778633118
Accuracy : 0.5
17 loss : 0.6930561423301697
Accuracy : 0.5
18 loss : 0.6637531280517578
Accuracy : 0.5
19 loss : 0.6487874627113343
Accuracy : 0.8
20 loss : 0.6442154169082641
Accuracy : 0.65
21 loss : 0.6168811917304993
Accuracy : 0.8
22 loss : 0.6067142486572266
Accuracy : 0.75
23 loss : 0.6123998761177063
Accuracy : 0.

In [23]:
mlp_model2 = SimpleMLP_Sigmoid().cuda()
train_loss_lst2 = []
test_accuracy_lst2 = []
for epoch in range(total_epoch):
  train_loss = train(mlp_model, train_loader)
  train_loss_lst2.append(train_loss)
  test_accuracy = eval(mlp_model, test_loader)
  test_accuracy_lst2.append(test_accuracy)
  
  print(epoch+1, "loss :", train_loss)
  print("Accuracy :", test_accuracy)
  
summary(mlp_model2, input_size = (3, 32, 32))

1 loss : 0.14922636449337007
Accuracy : 0.7
2 loss : 0.1759720951318741
Accuracy : 0.7
3 loss : 0.13643755167722701
Accuracy : 0.75
4 loss : 0.13886952251195908
Accuracy : 0.7
5 loss : 0.1154312402009964
Accuracy : 0.8
6 loss : 0.10226316004991531
Accuracy : 0.7
7 loss : 0.15439576804637908
Accuracy : 0.7
8 loss : 0.10304205864667892
Accuracy : 0.8
9 loss : 0.09635858982801437
Accuracy : 0.8
10 loss : 0.1065976932644844
Accuracy : 0.8
11 loss : 0.09248813986778259
Accuracy : 0.8
12 loss : 0.10646354258060456
Accuracy : 0.8
13 loss : 0.07999760508537293
Accuracy : 0.8
14 loss : 0.07754777148365974
Accuracy : 0.8
15 loss : 0.07992866188287735
Accuracy : 0.8
16 loss : 0.0788854070007801
Accuracy : 0.85
17 loss : 0.06269540265202522
Accuracy : 0.8
18 loss : 0.07590291909873485
Accuracy : 0.8
19 loss : 0.062147921323776244
Accuracy : 0.8
20 loss : 0.05436566770076752
Accuracy : 0.7
21 loss : 0.07661194577813149
Accuracy : 0.7
22 loss : 0.08266534693539143
Accuracy : 0.8
23 loss : 0.04468522